<a href="https://colab.research.google.com/github/orelanthos32/InfoRetrieval/blob/master/Bert_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install transformers

     |████████████████████████████████| 2.0MB 7.8MB/s 
     |████████████████████████████████| 890kB 36.9MB/s 
     |████████████████████████████████| 3.2MB 41.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=d2c41d8c8a0d466d0e67b7eb904b9348940bda9e4d46ec2e25f7003198b447cc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:

from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.summary()


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [6]:
import tensorflow as tf
import pandas as pd

In [1]:
from google.colab import files
uploaded = files.upload()

Saving stockticker10-tweets(5-3-21) label.csv to stockticker10-tweets(5-3-21) label.csv


In [50]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['stockticker10-tweets(5-3-21) label.csv']))
df2.head()

,Query,Datetime(UTC),Tweet ID,Text,Username,RT,RT Count,Fav Count,Sentiment (2pos/1neu/0neg),Subjectivity (1opinion/0fact)
0,$GME,5/3/2021 6:00,1.370000e+18,Think we're scared of the market makers?\n\n$A...,indigodisruptor,Y,9,0,NaN,NaN
1,$GME,5/3/2021 6:00,1.370000e+18,What these suits don’t know is $amc $gme wasn’...,Ladude2014,Y,5,0,NaN,NaN
2,$GME,5/3/2021 6:00,1.370000e+18,Holy shit LMFAO. \n$AMC $GME \n\nTRUST THE APE...,handsomephil5,Y,48,0,NaN,NaN
3,$GME,5/3/2021 6:00,1.370000e+18,"🔥 GUH: ""Fidelity closed out my shares of GME a...",WSBGains,N,0,0,0.0,1.0
4,$GME,5/3/2021 6:00,1.370000e+18,Amc Buy and hold Short Intrest borrow fee up t...,itsfibonacci96,Y,10,0,NaN,NaN


In [51]:
df2 = df2.dropna()

In [52]:
df2

,Query,Datetime(UTC),Tweet ID,Text,Username,RT,RT Count,Fav Count,Sentiment (2pos/1neu/0neg),Subjectivity (1opinion/0fact)
3,$GME,5/3/2021 6:00,1.370000e+18,"🔥 GUH: ""Fidelity closed out my shares of GME a...",WSBGains,N,0,0,0.0,1.0
6,$GME,5/3/2021 5:59,1.370000e+18,Astounding that the $GME July $800 Call was up...,ColtStock,N,0,0,2.0,1.0
21,$GME,5/3/2021 5:52,1.370000e+18,TD Ameritrade restricting Short Selling of $GM...,ajit_p_singh,N,1,1,0.0,1.0
27,$GME,5/3/2021 5:50,1.370000e+18,55k YOLO on $SLGG Activist investor coming in ...,0ptimusPrim0,N,1,1,2.0,1.0
36,$GME,5/3/2021 5:46,1.370000e+18,$GME vs other asset class since house financia...,Pointedmacro,N,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
9970,$AAPL,4/3/2021 22:59,1.370000e+18,The Nasdaq 100 index closed &gt;10% down from ...,diamondoptions2,N,1,13,1.0,0.0
9978,$AAPL,4/3/2021 22:56,1.370000e+18,SmartOptions® Unusual Activity Alert\n(Delayed...,SmartOptionsAi,N,0,0,1.0,0.0
9980,$AAPL,4/3/2021 22:56,1.370000e+18,Rising Yield &amp; Inflation fear are causing ...,realRajatDey,N,0,0,1.0,0.0
9983,$AAPL,4/3/2021 22:54,1.370000e+18,Large cap stocks with the most shares traded t...,StockMKTNewz,N,3,17,1.0,0.0


In [53]:
df2 = df2[['Text','Sentiment (2pos/1neu/0neg)']]

In [54]:
df2 = df2.rename(columns={"Sentiment (2pos/1neu/0neg)": "LABEL_COLUMN","Text" : "DATA_COLUMN"})

In [55]:
df2 = df2[df2.LABEL_COLUMN != 1]
df2.loc[df2.LABEL_COLUMN == 2, 'LABEL_COLUMN'] = 1

In [56]:
import numpy as np
msk = np.random.rand(len(df2)) < 0.8

train = df2[msk]

test = df2[~msk]

In [58]:
test.head()

,DATA_COLUMN,LABEL_COLUMN
6,Astounding that the $GME July $800 Call was up...,1.0
65,@markminervini Was there a problem? $GME did j...,0.0
94,Champion High Short Interest Stocks 3/4/2021:...,1.0
102,@GMEshortsqueeze I think esports is the angle ...,1.0
108,@linkmarine33 @coinbasebro There's some though...,1.0


In [59]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [38]:

def convert_data_to_examples(train, test, Text, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [60]:

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [61]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
     34/Unknown - 1310s 38s/step - loss: 0.6603 - accuracy: 0.6302WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
34/34 [==============================] - 1363s 39s/step - loss: 0.6597 - accuracy: 0.6313 - val_loss: 0.6293 - val_accuracy: 0.6765
Epoch 2/2
34/34 [==============================] - 1337s 39s/step - loss: 0.5809 - accuracy: 0.7279 - val_loss: 0.5918 - val_accuracy: 0.6838


In [ ]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']

In [ ]:

tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])